## Имя проекта - *Scrapping avito and forecast price*

## Цели:
- Скрапинг данных жилой недвижимости с сайта Avito.ru.
- Получение прогнозной цены жилого объекта недвижимости в зависимости от параметров.

## Задачи:
1. скрапинг данных с авито;
1. подготовка данных для работы;
1. проверка датасета;
1. подготовка параметров к моделированию;
1. обучение модели, проверка на тестовом множестве и корректировка модели;
1. получить прогнозную цену по параметрам объекта жилой недвижимости;

In [ ]:
from bs4 import BeautifulSoup
import requests
from requests import get
import time
import random
from hyper.contrib import HTTP20Adapter # Обход ошибки 403

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
HEADERS = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15'}

rooms = []
prices = []
areas = []
floors = []
max_floors = []
links = []
balcony_or_loggia = []
bathroom = []
built_year = []

In [ ]:
import time
startTime = time.time()

def main_request(url):
    '''Запрос страницы с сайта'''
    s = requests.Session()
    s.mount('https://', HTTP20Adapter())
    response = s.get(url, headers=HEADERS)
    return response


def select_blocks(url, tag, class_name):
    '''Отбор блоков объявлений'''
    html_soup = BeautifulSoup(main_request(url).content, 'html.parser')
    house_data = html_soup.find_all(tag, class_=class_name)
    return house_data


def select_features(house_data):
    '''Добавление признака в соответствующий список'''
    count_inner = 0
    while count_inner < 10: #len(house_data)    
        house = house_data[count_inner]

        # Определение стоимости квартиры
        price = house.find('span', {'class': 'price-text-_YGDY text-text-LurtD text-size-s-BxGpL'}).text
        price = ''.join(price.split()[:-1])

        # Сбор строчек с заголовка (кол-во комнат, метраж, этаж)
        if house.find('a', {'class': 'link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-listRedesign-_rejR title-root_maxHeight-X6PsH'}):
            title = house.find('a', {'class': 'link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-listRedesign-_rejR title-root_maxHeight-X6PsH'}).text
        elif house.find('a', {'class': 'link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-large-jkTpZ title-root_maxHeight-X6PsH'}):
            title = house.find('a', {'class': 'link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-large-jkTpZ title-root_maxHeight-X6PsH'}).text
        else:
            print('Ошибка поиска title')
        title = [i.strip(',') for i in title.split()]

        # Добавление признаков в соответсвующие списки
        if title[1] == 'квартира':                  # Если заголовок имеет кол-во комнат
            rooms.append(title[0][0])
            area = '.'.join(title[2].split(','))
            floor = title[4].split('/')[0]
            max_floor = title[4].split('/')[1]
        else:                                       # Если заголовок "квартира-студия"
            rooms.append(1)                         # Если квартира-студия, то пусть комнат будет 1
            area = '.'.join(title[1].split(','))

        prices.append(price)
        areas.append(area)
        floors.append(floor)
        max_floors.append(max_floor)

        link = 'https://www.avito.ru' + house.find('a')['href']
        links.append(link)
        
        # Запрос со страницы объекта недвижимости и отбор признаков
        #delay()
        house_info = select_blocks(link, page_tag, page_class_name)
        
        
        bal_or_log = ''
        bath = ''
        year = ''
        for i in house_info:
            if 'Балкон' in i.find('span', {'class': 'item-params-label'}).text:
                bal_or_log = i.text.split()[-1]
            if 'Санузел' in i.find('span', {'class': 'item-params-label'}).text:
                bath = i.text.split()[-1]
            if 'Год' in i.find('span', {'class': 'item-params-label'}).text:
                year = i.text.split()[-1]
        if bal_or_log:
            balcony_or_loggia.append(bal_or_log)
        else:
            balcony_or_loggia.append(np.NaN)
        if bath:
            bathroom.append(bath)
        else:
            bathroom.append(np.NaN)
        if year:
            built_year.append(year)
        else:
            built_year.append(np.NaN)
        
        count_inner += 1
        return None
        
def delay():
    '''Добавление задержки перед следующим запросом'''
    value = random.random()
    scaled_value = 1 + (value * (9 - 7))
    #print(scaled_value)
    time.sleep(scaled_value)
    return None


main_tag = 'div'
main_class_name = 'iva-item-content-rejJg'
page_tag = 'li'
page_class_name = 'item-params-list-item'
count = 1

while count <= 1: 
    url = 'https://www.avito.ru/perm/kvartiry/prodam/vtorichka-ASgBAgICAkSSA8YQ5geMUg?context=H4sIAAAAAAAA_0q0MrSqLraysFJKK8rPDUhMT1WyLrYyNLNSKk5NLErOcMsvyg3PTElPLVGyrgUEAAD__xf8iH4tAAAA&p=' + str(count)
    select_features(select_blocks(url, main_tag, main_class_name))
    count += 1

endTime = time.time() 
totalTime = endTime - startTime 
print("Время выполнения блока = ", totalTime)

dataset = pd.read_csv('dataset.csv')
def update_df():
    count = 1
    while count <= 1: 
        url = 'https://www.avito.ru/perm/kvartiry/prodam/vtorichka-ASgBAgICAkSSA8YQ5geMUg?context=H4sIAAAAAAAA_0q0MrSqLraysFJKK8rPDUhMT1WyLrYyNLNSKk5NLErOcMsvyg3PTElPLVGyrgUEAAD__xf8iH4tAAAA&p=' + str(count)
        select_features(select_blocks(url, main_tag, main_class_name))
        update_df = pd.DataFrame({'Price': prices, 
                                   'Rooms': rooms, 
                                   'Area': areas, 
                                   'Floor': floors, 
                                   'Max_house_floor': max_floors, 
                                   'balcony\loggia': balcony_or_loggia,
                                   'Bathroom': bathroom,
                                   'Year built': built_year,
                                   'Link': links})
        for i in range(len(update_df.Link)):
            if update_df.Link[i] in dataset.Link:
                continue
            else:
                dataset = pd.concat(dataset, update_df.iloc[[i]])
        count += 1

update_df()

In [ ]:
df = pd.DataFrame({'Price': prices, 
                   'Rooms': rooms, 
                   'Area': areas, 
                   'Floor': floors, 
                   'Max_house_floor': max_floors, 
                   'balcony\loggia': balcony_or_loggia,
                   'Bathroom': bathroom,
                   'Year built': built_year,
                   'Link': links})

In [ ]:
dataset.shape

In [ ]:
df.head()

In [ ]:
for i in df.Area:
    try:
        float(i)
    except:
        print(i)

In [ ]:
df.query('Area == "в" or Area ==  "планировка"')

In [ ]:
df = df.drop([68, 398], axis=0)
df.shape

In [ ]:
df.dtypes

In [ ]:
df.Area = df.Area.astype(float)

In [ ]:
for column in df.columns:
    print(column + ':')
    print(df[column].isnull().value_counts(), end='\n\n')

In [ ]:
X = df.iloc[:, 1:5]
y = df.Price

In [ ]:
sns.regplot(x='Area' ,y='Price', data=df)
plt.xlim(0, 80)
plt.ylim(0, 1e7)

In [ ]:
sns.displot(x='Rooms', data=df)
plt.xlim(1, 5)

In [ ]:
sns.heatmap(df.corr(), annot=True)